In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import warnings


warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('sales_data_sample.csv',sep=",",encoding='Latin-1')
df.head(5)

ORDERNUMBER  QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER    SALES  \
0        10107               30      95.70                2  2871.00   
1        10121               34      81.35                5  2765.90   
2        10134               41      94.74                2  3884.34   
3        10145               45      83.26                6  3746.70   
4        10159               49     100.00               14  5205.27   

         ORDERDATE   STATUS  QTR_ID  MONTH_ID  YEAR_ID  ...  \
0   2/24/2003 0:00  Shipped       1         2     2003  ...   
1    5/7/2003 0:00  Shipped       2         5     2003  ...   
2    7/1/2003 0:00  Shipped       3         7     2003  ...   
3   8/25/2003 0:00  Shipped       3         8     2003  ...   
4  10/10/2003 0:00  Shipped       4        10     2003  ...   

                    ADDRESSLINE1  ADDRESSLINE2           CITY STATE  \
0        897 Long Airport Avenue           NaN            NYC    NY   
1             59 rue de l'Abbaye           NaN          Reims   NaN   
2  27 rue du Colonel Pierre Avia           NaN          Paris   NaN   
3             78934 Hillside Dr.           NaN       Pasadena    CA   
4                7734 Strong St.           NaN  San Francisco    CA   

  POSTALCODE COUNTRY TERRITORY CONTACTLASTNAME CONTACTFIRSTNAME DEALSIZE  
0      10022     USA       NaN              Yu             Kwai    Small  
1      51100  France      EMEA         Henriot             Paul    Small  
2      75508  France      EMEA        Da Cunha           Daniel   Medium  
3      90003     USA       NaN           Young            Julie   Medium  
4        NaN     USA       NaN           Brown            Julie   Medium  

[5 rows x 25 columns]

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2823 non-null   int64  
 1   QUANTITYORDERED   2823 non-null   int64  
 2   PRICEEACH         2823 non-null   float64
 3   ORDERLINENUMBER   2823 non-null   int64  
 4   SALES             2823 non-null   float64
 5   ORDERDATE         2823 non-null   object 
 6   STATUS            2823 non-null   object 
 7   QTR_ID            2823 non-null   int64  
 8   MONTH_ID          2823 non-null   int64  
 9   YEAR_ID           2823 non-null   int64  
 10  PRODUCTLINE       2823 non-null   object 
 11  MSRP              2823 non-null   int64  
 12  PRODUCTCODE       2823 non-null   object 
 13  CUSTOMERNAME      2823 non-null   object 
 14  PHONE             2823 non-null   object 
 15  ADDRESSLINE1      2823 non-null   object 
 16  ADDRESSLINE2      302 non-null    object 


In [21]:
df.describe()

ORDERNUMBER  QUANTITYORDERED    PRICEEACH  ORDERLINENUMBER  \
count   2823.000000      2823.000000  2823.000000      2823.000000   
mean   10258.725115        35.092809    83.658544         6.466171   
std       92.085478         9.741443    20.174277         4.225841   
min    10100.000000         6.000000    26.880000         1.000000   
25%    10180.000000        27.000000    68.860000         3.000000   
50%    10262.000000        35.000000    95.700000         6.000000   
75%    10333.500000        43.000000   100.000000         9.000000   
max    10425.000000        97.000000   100.000000        18.000000   

              SALES       QTR_ID     MONTH_ID     YEAR_ID         MSRP  
count   2823.000000  2823.000000  2823.000000  2823.00000  2823.000000  
mean    3553.889072     2.717676     7.092455  2003.81509   100.715551  
std     1841.865106     1.203878     3.656633     0.69967    40.187912  
min      482.130000     1.000000     1.000000  2003.00000    33.000000  
25%     2203.430000     2.000000     4.000000  2003.00000    68.000000  
50%     3184.800000     3.000000     8.000000  2004.00000    99.000000  
75%     4508.000000     4.000000    11.000000  2004.00000   124.000000  
max    14082.800000     4.000000    12.000000  2005.00000   214.000000

In [22]:
df.isnull().sum()

ORDERNUMBER            0
QUANTITYORDERED        0
PRICEEACH              0
ORDERLINENUMBER        0
SALES                  0
ORDERDATE              0
STATUS                 0
QTR_ID                 0
MONTH_ID               0
YEAR_ID                0
PRODUCTLINE            0
MSRP                   0
PRODUCTCODE            0
CUSTOMERNAME           0
PHONE                  0
ADDRESSLINE1           0
ADDRESSLINE2        2521
CITY                   0
STATE               1486
POSTALCODE            76
COUNTRY                0
TERRITORY           1074
CONTACTLASTNAME        0
CONTACTFIRSTNAME       0
DEALSIZE               0
dtype: int64

In [23]:

df.shape

(2823, 25)

In [24]:
# concating MSRP and PRODUCT
df['MSRP_Product'] = df['MSRP'].astype(str) + df['PRODUCTCODE']

# dropping irrelevant columns
columns_to_drop = ['TERRITORY','STATE','ADDRESSLINE2','ADDRESSLINE1','PHONE','POSTALCODE',
                   'MSRP','PRODUCTCODE','CONTACTFIRSTNAME','CONTACTLASTNAME','ORDERDATE','ORDERNUMBER']
df.drop(columns=columns_to_drop, axis=1, inplace=True)

# changing datatypes from numeric to string 
df.QTR_ID = df.QTR_ID.astype(str)
df.MONTH_ID = df.MONTH_ID.astype(str)
df.YEAR_ID = df.YEAR_ID.astype(str)
df.ORDERLINENUMBER = df.ORDERLINENUMBER.astype(str)

df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   QUANTITYORDERED  2823 non-null   int64  
 1   PRICEEACH        2823 non-null   float64
 2   ORDERLINENUMBER  2823 non-null   object 
 3   SALES            2823 non-null   float64
 4   STATUS           2823 non-null   object 
 5   QTR_ID           2823 non-null   object 
 6   MONTH_ID         2823 non-null   object 
 7   YEAR_ID          2823 non-null   object 
 8   PRODUCTLINE      2823 non-null   object 
 9   CUSTOMERNAME     2823 non-null   object 
 10  CITY             2823 non-null   object 
 11  COUNTRY          2823 non-null   object 
 12  DEALSIZE         2823 non-null   object 
 13  MSRP_Product     2823 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 308.9+ KB


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   QUANTITYORDERED  2823 non-null   int64  
 1   PRICEEACH        2823 non-null   float64
 2   ORDERLINENUMBER  2823 non-null   object 
 3   SALES            2823 non-null   float64
 4   STATUS           2823 non-null   object 
 5   QTR_ID           2823 non-null   object 
 6   MONTH_ID         2823 non-null   object 
 7   YEAR_ID          2823 non-null   object 
 8   PRODUCTLINE      2823 non-null   object 
 9   CUSTOMERNAME     2823 non-null   object 
 10  CITY             2823 non-null   object 
 11  COUNTRY          2823 non-null   object 
 12  DEALSIZE         2823 non-null   object 
 13  MSRP_Product     2823 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 308.9+ KB


In [26]:
df

QUANTITYORDERED  PRICEEACH ORDERLINENUMBER    SALES    STATUS QTR_ID  \
0                  30      95.70               2  2871.00   Shipped      1   
1                  34      81.35               5  2765.90   Shipped      2   
2                  41      94.74               2  3884.34   Shipped      3   
3                  45      83.26               6  3746.70   Shipped      3   
4                  49     100.00              14  5205.27   Shipped      4   
...               ...        ...             ...      ...       ...    ...   
2818               20     100.00              15  2244.40   Shipped      4   
2819               29     100.00               1  3978.51   Shipped      1   
2820               43     100.00               4  5417.57  Resolved      1   
2821               34      62.24               1  2116.16   Shipped      1   
2822               47      65.52               9  3079.44   On Hold      2   

     MONTH_ID YEAR_ID  PRODUCTLINE              CUSTOMERNAME           CITY  \
0           2    2003  Motorcycles         Land of Toys Inc.            NYC   
1           5    2003  Motorcycles        Reims Collectables          Reims   
2           7    2003  Motorcycles           Lyon Souveniers          Paris   
3           8    2003  Motorcycles         Toys4GrownUps.com       Pasadena   
4          10    2003  Motorcycles  Corporate Gift Ideas Co.  San Francisco   
...       ...     ...          ...                       ...            ...   
2818       12    2004        Ships     Euro Shopping Channel         Madrid   
2819        1    2005        Ships   Oulu Toy Supplies, Inc.           Oulu   
2820        3    2005        Ships     Euro Shopping Channel         Madrid   
2821        3    2005        Ships              Alpha Cognac       Toulouse   
2822        5    2005        Ships         Gifts4AllAges.com         Boston   

      COUNTRY DEALSIZE MSRP_Product  
0         USA    Small   95S10_1678  
1      France    Small   95S10_1678  
2      France   Medium   95S10_1678  
3         USA   Medium   95S10_1678  
4         USA   Medium   95S10_1678  
...       ...      ...          ...  
2818    Spain    Small   54S72_3212  
2819  Finland   Medium   54S72_3212  
2820    Spain   Medium   54S72_3212  
2821   France    Small   54S72_3212  
2822      USA   Medium   54S72_3212  

[2823 rows x 14 columns]

In [27]:
df.to_csv('SalesDataMixed.csv', index=False) #this will be used for Kprototype modelling

## Dummy Transformation

In [14]:
cols = df.columns

num_cols = df._get_numeric_data().columns

num_cols

cat_cols = list(set(cols) - set(num_cols))

In [15]:
df_dummy = pd.get_dummies(data=df, columns=cat_cols, drop_first=True)
df_dummy

QUANTITYORDERED  PRICEEACH    SALES  ORDERLINENUMBER_10  \
0                  30      95.70  2871.00                   0   
1                  34      81.35  2765.90                   0   
2                  41      94.74  3884.34                   0   
3                  45      83.26  3746.70                   0   
4                  49     100.00  5205.27                   0   
...               ...        ...      ...                 ...   
2818               20     100.00  2244.40                   0   
2819               29     100.00  3978.51                   0   
2820               43     100.00  5417.57                   0   
2821               34      62.24  2116.16                   0   
2822               47      65.52  3079.44                   0   

      ORDERLINENUMBER_11  ORDERLINENUMBER_12  ORDERLINENUMBER_13  \
0                      0                   0                   0   
1                      0                   0                   0   
2                      0                   0                   0   
3                      0                   0                   0   
4                      0                   0                   0   
...                  ...                 ...                 ...   
2818                   0                   0                   0   
2819                   0                   0                   0   
2820                   0                   0                   0   
2821                   0                   0                   0   
2822                   0                   0                   0   

      ORDERLINENUMBER_14  ORDERLINENUMBER_15  ORDERLINENUMBER_16  ...  \
0                      0                   0                   0  ...   
1                      0                   0                   0  ...   
2                      0                   0                   0  ...   
3                      0                   0                   0  ...   
4                      1                   0                   0  ...   
...                  ...                 ...                 ...  ...   
2818                   0                   1                   0  ...   
2819                   0                   0                   0  ...   
2820                   0                   0                   0  ...   
2821                   0                   0                   0  ...   
2822                   0                   0                   0  ...   

      MONTH_ID_7  MONTH_ID_8  MONTH_ID_9  DEALSIZE_Medium  DEALSIZE_Small  \
0              0           0           0                0               1   
1              0           0           0                0               1   
2              1           0           0                1               0   
3              0           1           0                1               0   
4              0           0           0                1               0   
...          ...         ...         ...              ...             ...   
2818           0           0           0                0               1   
2819           0           0           0                1               0   
2820           0           0           0                1               0   
2821           0           0           0                0               1   
2822           0           0           0                1               0   

      QTR_ID_2  QTR_ID_3  QTR_ID_4  YEAR_ID_2004  YEAR_ID_2005  
0            0         0         0             0             0  
1            1         0         0             0             0  
2            0         1         0             0             0  
3            0         1         0             0             0  
4            0         0         1             0             0  
...        ...       ...       ...           ...           ...  
2818         0         0         1             1             0  
2819         0         0         0             0             1  
2820         0       

In [17]:
list(df_dummy.columns)

['QUANTITYORDERED',
 'PRICEEACH',
 'SALES',
 'ORDERLINENUMBER_10',
 'ORDERLINENUMBER_11',
 'ORDERLINENUMBER_12',
 'ORDERLINENUMBER_13',
 'ORDERLINENUMBER_14',
 'ORDERLINENUMBER_15',
 'ORDERLINENUMBER_16',
 'ORDERLINENUMBER_17',
 'ORDERLINENUMBER_18',
 'ORDERLINENUMBER_2',
 'ORDERLINENUMBER_3',
 'ORDERLINENUMBER_4',
 'ORDERLINENUMBER_5',
 'ORDERLINENUMBER_6',
 'ORDERLINENUMBER_7',
 'ORDERLINENUMBER_8',
 'ORDERLINENUMBER_9',
 'PRODUCTLINE_Motorcycles',
 'PRODUCTLINE_Planes',
 'PRODUCTLINE_Ships',
 'PRODUCTLINE_Trains',
 'PRODUCTLINE_Trucks and Buses',
 'PRODUCTLINE_Vintage Cars',
 'MSRP_Product_100S700_3505',
 'MSRP_Product_101S18_2949',
 'MSRP_Product_101S700_2824',
 'MSRP_Product_102S18_1342',
 'MSRP_Product_102S32_4485',
 'MSRP_Product_104S18_3136',
 'MSRP_Product_105S18_3856',
 'MSRP_Product_107S24_3432',
 'MSRP_Product_109S24_1785',
 'MSRP_Product_112S24_1578',
 'MSRP_Product_115S12_4675',
 'MSRP_Product_115S50_1392',
 'MSRP_Product_116S18_1097',
 'MSRP_Product_117S12_3380',
 'MSRP_

In [18]:
# Saving data transformed
df_dummy.to_csv('SalesDummyData.csv',index=False)

###### K-Prototype Implementation

In [3]:
df = pd.read_csv('SalesDataMixed.csv')
df

QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER    SALES    STATUS  QTR_ID  \
0                  30      95.70                2  2871.00   Shipped       1   
1                  34      81.35                5  2765.90   Shipped       2   
2                  41      94.74                2  3884.34   Shipped       3   
3                  45      83.26                6  3746.70   Shipped       3   
4                  49     100.00               14  5205.27   Shipped       4   
...               ...        ...              ...      ...       ...     ...   
2818               20     100.00               15  2244.40   Shipped       4   
2819               29     100.00                1  3978.51   Shipped       1   
2820               43     100.00                4  5417.57  Resolved       1   
2821               34      62.24                1  2116.16   Shipped       1   
2822               47      65.52                9  3079.44   On Hold       2   

      MONTH_ID  YEAR_ID  PRODUCTLINE              CUSTOMERNAME           CITY  \
0            2     2003  Motorcycles         Land of Toys Inc.            NYC   
1            5     2003  Motorcycles        Reims Collectables          Reims   
2            7     2003  Motorcycles           Lyon Souveniers          Paris   
3            8     2003  Motorcycles         Toys4GrownUps.com       Pasadena   
4           10     2003  Motorcycles  Corporate Gift Ideas Co.  San Francisco   
...        ...      ...          ...                       ...            ...   
2818        12     2004        Ships     Euro Shopping Channel         Madrid   
2819         1     2005        Ships   Oulu Toy Supplies, Inc.           Oulu   
2820         3     2005        Ships     Euro Shopping Channel         Madrid   
2821         3     2005        Ships              Alpha Cognac       Toulouse   
2822         5     2005        Ships         Gifts4AllAges.com         Boston   

      COUNTRY DEALSIZE MSRP_Product  
0         USA    Small   95S10_1678  
1      France    Small   95S10_1678  
2      France   Medium   95S10_1678  
3         USA   Medium   95S10_1678  
4         USA   Medium   95S10_1678  
...       ...      ...          ...  
2818    Spain    Small   54S72_3212  
2819  Finland   Medium   54S72_3212  
2820    Spain   Medium   54S72_3212  
2821   France    Small   54S72_3212  
2822      USA   Medium   54S72_3212  

[2823 rows x 14 columns]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   QUANTITYORDERED  2823 non-null   int64  
 1   PRICEEACH        2823 non-null   float64
 2   ORDERLINENUMBER  2823 non-null   int64  
 3   SALES            2823 non-null   float64
 4   STATUS           2823 non-null   object 
 5   QTR_ID           2823 non-null   int64  
 6   MONTH_ID         2823 non-null   int64  
 7   YEAR_ID          2823 non-null   int64  
 8   PRODUCTLINE      2823 non-null   object 
 9   CUSTOMERNAME     2823 non-null   object 
 10  CITY             2823 non-null   object 
 11  COUNTRY          2823 non-null   object 
 12  DEALSIZE         2823 non-null   object 
 13  MSRP_Product     2823 non-null   object 
dtypes: float64(2), int64(5), object(7)
memory usage: 308.9+ KB
